## Įvadas 

Ši pamoka apims: 
- Kas yra funkcijos kvietimas ir jo panaudojimo atvejai 
- Kaip sukurti funkcijos kvietimą naudojant OpenAI 
- Kaip integruoti funkcijos kvietimą į programą 

## Mokymosi tikslai 

Baigę šią pamoką žinosite ir suprasite: 

- Funkcijos kvietimo naudojimo paskirtį 
- Kaip nustatyti funkcijos kvietimą naudojant OpenAI paslaugą 
- Kaip sukurti efektyvius funkcijos kvietimus jūsų programos naudojimo atvejui 


## Funkcijų kvietimų supratimas

Šiam pamokai norime sukurti funkciją mūsų švietimo startuoliui, leidžiančią vartotojams naudoti pokalbių robotą techninių kursų paieškai. Rekomenduosime kursus, atitinkančius jų įgūdžių lygį, dabartinę pareigybę ir dominančią technologiją.

Norėdami tai įgyvendinti, naudosime šių derinį:
 - `OpenAI`, kad sukurtume vartotojui pokalbių patirtį
 - `Microsoft Learn Catalog API`, kad padėtume vartotojams rasti kursus pagal jų užklausą
 - `Function Calling`, kad vartotojo užklausą perduotume funkcijai, kuri atliks API užklausą.

Pradėkime nuo to, kodėl iš viso norėtume naudoti funkcijų kvietimą:

print("Žinutės kitoje užklausoje:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # gauti naują atsakymą iš GPT, kuriame matomas funkcijos atsakymas


print(second_response.choices[0].message)


### Kodėl funkcijų kvietimas

Jei jau baigėte bet kurį kitą šio kurso pamoką, tikriausiai suprantate didelių kalbos modelių (LLM) naudojimo galią. Tikimės, kad taip pat matote kai kurias jų ribotumas.

Funkcijų kvietimas yra OpenAI paslaugos funkcija, skirta spręsti šias problemas:

Nenuosekli atsakymų formatavimo:
- Prieš funkcijų kvietimą, didelio kalbos modelio atsakymai buvo nestruktūruoti ir nenuoseklūs. Kūrėjams reikėjo rašyti sudėtingą validacijos kodą, kad apdorotų kiekvieną išvesties variaciją.

Ribota integracija su išoriniais duomenimis:
- Prieš šią funkciją buvo sunku įtraukti duomenis iš kitų programos dalių į pokalbio kontekstą.

Standartizuodama atsakymų formatus ir leidžiant sklandžią integraciją su išoriniais duomenimis, funkcijų kvietimas supaprastina kūrimą ir sumažina papildomos validacijos logikos poreikį.

Vartotojai negalėjo gauti atsakymų, tokių kaip „Koks yra dabartinis oras Stokholme?“. Tai todėl, kad modeliai buvo apriboti duomenų mokymo laiku.

Pažiūrėkime žemiau pateiktą pavyzdį, kuris iliustruoja šią problemą:

Tarkime, norime sukurti studentų duomenų bazę, kad galėtume jiems pasiūlyti tinkamą kursą. Žemiau turime du studentų aprašymus, kurie yra labai panašūs pagal turimus duomenis.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Mes norime tai siųsti LLM, kad jis išanalizuotų duomenis. Tai vėliau gali būti naudojama mūsų programoje, norint siųsti tai į API arba saugoti duomenų bazėje.

Sukurkime du identiškus užklausimus, kuriuose nurodome LLM, kokia informacija mus domina:


Mes norime tai siųsti LLM, kad jis išskirtų mūsų produktui svarbias dalis. Taigi galime sukurti du identiškus užklausimus, kad nurodytume LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Sukūrę šiuos du užklausimus, mes juos išsiųsime LLM naudodami `openai.ChatCompletion`. Užklausimą saugome kintamajame `messages` ir priskiriame vaidmenį `user`. Tai imituoja vartotojo žinutės rašymą pokalbių robotui.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Dabar galime išsiųsti abu užklausimus LLM ir išnagrinėti gautą atsakymą.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Nors užklausos yra tos pačios ir aprašymai panašūs, mes galime gauti skirtingus `Grades` savybės formatus.

Jei aukščiau pateiktą langelį paleisite kelis kartus, formatas gali būti `3.7` arba `3.7 GPA`.

Taip yra todėl, kad LLM priima nestruktūruotus duomenis rašytinės užklausos pavidalu ir taip pat grąžina nestruktūruotus duomenis. Mums reikia turėti struktūruotą formatą, kad žinotume, ko tikėtis saugant ar naudojant šiuos duomenis.

Naudodami funkcijų kvietimą galime užtikrinti, kad gausime struktūruotus duomenis atgal. Naudojant funkcijų kvietimą, LLM iš tikrųjų nekviečia ar nevykdo jokių funkcijų. Vietoj to, mes sukuriame struktūrą, kurios LLM turi laikytis savo atsakymuose. Tada naudojame tuos struktūruotus atsakymus, kad žinotume, kurią funkciją vykdyti mūsų programose.


![Funkcijų kvietimo srauto diagrama](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.lt.png)


Tada galime paimti tai, kas grąžinama iš funkcijos, ir siųsti tai atgal į LLM. LLM tuomet atsakys natūralia kalba, kad atsakytų į vartotojo užklausą.


### Naudojimo atvejai funkcijų kvietimams

**Išorinių įrankių kvietimas**  
Pokalbių robotai puikiai atsako į vartotojų klausimus. Naudodami funkcijų kvietimus, pokalbių robotai gali naudoti vartotojų žinutes tam tikroms užduotims atlikti. Pavyzdžiui, studentas gali paprašyti pokalbių roboto „Išsiųsk el. laišką mano dėstytojui, sakydamas, kad man reikia daugiau pagalbos su šia tema“. Tai gali sukelti funkcijos kvietimą `send_email(to: string, body: string)`.

**API arba duomenų bazės užklausų kūrimas**  
Vartotojai gali rasti informaciją naudodami natūralią kalbą, kuri paverčiama į suformatuotą užklausą arba API užklausą. Pavyzdys galėtų būti mokytojas, kuris prašo „Kas yra studentai, kurie atliko paskutinį užduotį“, ir tai gali iškviesti funkciją pavadinimu `get_completed(student_name: string, assignment: int, current_status: string)`.

**Struktūruotų duomenų kūrimas**  
Vartotojai gali paimti teksto bloką arba CSV ir naudoti LLM, kad ištrauktų svarbią informaciją. Pavyzdžiui, studentas gali konvertuoti Vikipedijos straipsnį apie taikos susitarimus, kad sukurtų AI atmintines. Tai galima padaryti naudojant funkciją `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. Pirmojo funkcijos kvietimo kūrimas

Funkcijos kvietimo kūrimo procesas apima 3 pagrindinius žingsnius:
1. Iškvietimas Chat Completions API su jūsų funkcijų sąrašu ir vartotojo žinute
2. Modelio atsakymo skaitymas, kad būtų atliktas veiksmas, t.y. įvykdyta funkcija arba API kvietimas
3. Kitas kvietimas Chat Completions API su jūsų funkcijos atsakymu, kad būtų panaudota ši informacija atsakymui vartotojui sukurti.


![Funkcijos kvietimo srautas](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.lt.png)


### Funkcijos kvietimo elementai

#### Vartotojo įvestis

Pirmasis žingsnis yra sukurti vartotojo žinutę. Tai gali būti dinamiškai priskirta paimant teksto įvesties reikšmę arba galite priskirti reikšmę čia. Jei tai pirmas kartas, kai dirbate su Chat Completions API, turime apibrėžti žinutės `role` ir `content`.

`role` gali būti `system` (taisyklių kūrimas), `assistant` (modelis) arba `user` (galutinis vartotojas). Funkcijų kvietimui mes priskirsime `user` ir pateiksime pavyzdinį klausimą.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funkcijų kūrimas.

Toliau apibrėšime funkciją ir tos funkcijos parametrus. Čia naudosime tik vieną funkciją, vadinamą `search_courses`, tačiau galite sukurti kelias funkcijas.

**Svarbu**: Funkcijos įtraukiamos į sistemos pranešimą LLM ir bus įskaičiuotos į turimų žetonų kiekį.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Apibrėžimai** 

Funkcijos apibrėžimo struktūra turi kelis lygius, kiekvienas su savo savybėmis. Štai išskaidymas įdėtoje struktūroje:

**Pagrindinės funkcijos savybės:**

`name` - Funkcijos pavadinimas, kurią norime iškviesti. 

`description` - Tai aprašymas, kaip funkcija veikia. Čia svarbu būti konkrečiam ir aiškiam. 

`parameters` - Reikšmių ir formato sąrašas, kurį norite, kad modelis pateiktų savo atsakyme. 

**Parametrų objekto savybės:**

`type` - Parametrų objekto duomenų tipas (dažniausiai "object")

`properties` - Konkretūs reikšmių sąrašai, kuriuos modelis naudos savo atsakyme. 

**Individualių parametrų savybės:**

`name` - Implicitškai apibrėžtas pagal savybės raktą (pvz., "role", "product", "level")

`type` - Šio konkretaus parametro duomenų tipas (pvz., "string", "number", "boolean") 

`description` - Konkretus parametro aprašymas. 

**Pasirenkamos savybės:**

`required` - Masyvas, nurodantis, kurie parametrai yra privalomi, kad funkcijos kvietimas būtų įvykdytas. 


### Funkcijos kvietimo atlikimas  
Apibrėžus funkciją, dabar turime ją įtraukti į kvietimą Chat Completion API. Tai darome pridėdami `functions` prie užklausos. Šiuo atveju `functions=functions`.  

Taip pat yra galimybė nustatyti `function_call` į `auto`. Tai reiškia, kad leisime LLM nuspręsti, kuri funkcija turėtų būti kviečiama pagal vartotojo žinutę, o ne priskirsime ją patys.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Dabar pažvelkime į atsakymą ir pamatykime, kaip jis yra suformatuotas:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Matote, kad yra iškviestas funkcijos pavadinimas, o iš vartotojo žinutės LLM sugebėjo rasti duomenis, atitinkančius funkcijos argumentus.


## 3.Funkcijų kvietimų integravimas į programą. 


Kai išbandėme suformatuotą atsakymą iš LLM, dabar galime tai integruoti į programą. 

### Srauto valdymas 

Norėdami tai integruoti į mūsų programą, atlikime šiuos veiksmus: 

Pirmiausia atlikime kvietimą OpenAI paslaugoms ir išsaugokime žinutę kintamajame, pavadintame `response_message`. 


In [ ]:
response_message = response.choices[0].message

Dabar apibrėšime funkciją, kuri kvies Microsoft Learn API, kad gautų kursų sąrašą:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Kaip geriausia praktika, mes tada patikrinsime, ar modelis nori iškviesti funkciją. Po to sukursime vieną iš galimų funkcijų ir suderinsime ją su kviečiama funkcija.  
Tada paimsime funkcijos argumentus ir susiesime juos su argumentais iš LLM.

Galiausiai pridėsime funkcijos kvietimo žinutę ir reikšmes, kurias grąžino `search_courses` žinutė. Tai suteikia LLM visą reikalingą informaciją, kad jis galėtų natūralia kalba atsakyti vartotojui.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Dabar mes išsiųsime atnaujintą žinutę į LLM, kad galėtume gauti natūralios kalbos atsakymą, o ne API JSON formato atsakymą.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodo iššūkis 

Puikus darbas! Norėdami tęsti mokymąsi apie OpenAI funkcijų kvietimą, galite sukurti: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Daugiau funkcijos parametrų, kurie gali padėti besimokantiesiems rasti daugiau kursų. Galimus API parametrus galite rasti čia:  
 - Sukurkite kitą funkcijos kvietimą, kuris priimtų daugiau informacijos iš besimokančiojo, pavyzdžiui, jų gimtąją kalbą  
 - Sukurkite klaidų tvarkymą, kai funkcijos kvietimas ir/arba API kvietimas negrąžina tinkamų kursų  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojamas profesionalus žmogaus vertimas. Mes neatsakome už bet kokius nesusipratimus ar neteisingus aiškinimus, kilusius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
